# Patent citation 제작

### 1. patent assignee, patent year 제작
- 기존 파일에 옛날 patent 정보 (3,4,5번대) 업데이트 필요

### 2. 기업간 adj matrix 제작
- (patent_id -> cited)를 (gvkey -> gvkey)로 바꾸기

### 3. weight sum
- 전에 구한 weight로 weight sum해서 firm level 변수 제작

### 4. patent stock
- 01~05부터 5년 단위로 잘라서 07~11까지 firm level 변수 제작

### 1. patent assignee 제작
- __6836904번까지 assignee 정보, year 정보 필요__
- assingee
    - raw data에서 patent_id, raw_assignee 추출  
    - raw_assignee -> assignee (assignee_raw join)
    - 기존 csv에 합치기 (pd.concat)
- year
    - nber 기준 2001 ~ 2004 patent 정보 뽑기 (pid, nber, uspc, uspc_sub, year)
    - 기존 csv에 합치기 (pd.concat)

__patent_id, raw_assignee 추출__

In [1]:
import os
import csv
import zipfile

os.chdir("E:/patent/all_patent")

In [2]:
zf_patent1 = zipfile.ZipFile('harvard_dataset_final_merged_output_1.zip') 
csv_patent1 = zf_patent1.open('harvard_dataset_final_merged_output_1.csv')
csv_patent_reader1 = csv.reader(csv_patent1)
header = csv_patent_reader1.next()

zf_patent2 = zipfile.ZipFile('harvard_dataset_final_merged_output_2.zip') 
csv_patent2 = zf_patent2.open('harvard_dataset_final_merged_output_2.csv')
csv_patent_reader2 = csv.reader(csv_patent2)
header = csv_patent_reader2.next()

In [3]:
#select the assignee col
assignee_col = []
for i in range(0,len(header)):
    if 'organization' in header[i]:   
        assignee_col.append(i)
        
del assignee_col[-3:]    #delete lawyer org

In [4]:
assignee = []
p_id = []

for line in csv_patent_reader1:
    for i in assignee_col:
        if line[i]=='':
            break
        
        elif line[i]=='nan':
            assignee.append('private')
            p_id.append(line[1])
                        
        else:
            assignee.append(line[i])
            p_id.append(line[1])
                        
for line in csv_patent_reader2:
    if line[i] == '6836905':
        break
    for i in assignee_col:
        if line[i]=='':
            break
        
        elif line[i]=='nan':
            assignee.append('private')
            p_id.append(line[1])
                        
        else:
            assignee.append(line[i])
            p_id.append(line[1])

NameError: name 'pd' is not defined

In [5]:
len(p_id)

6894521

In [9]:
p_id[-1]

'RE45078'

In [10]:
import pandas as pd

df = pd.DataFrame({'patent_id':p_id,'raw_assignee':assignee})
df = df.drop_duplicates()

In [11]:
len(df)

5002108

In [12]:
df.sort_values(by='patent_id')
df.iloc[0:10,:]

,patent_id,raw_assignee
0,3858241,"UNITED STATES OF AMERICA, ARMY"
1,3858242,private
2,3858243,PLYMATIC
3,3858244,private
4,3858245,HAIR AGAIN LTD.
5,3858246,private
6,3858247,private
7,3858248,private
8,3858249,private
9,3858250,private


In [13]:
df = df.loc[df['raw_assignee']!='private',:]
len(df)

4641719

In [18]:
df.reset_index(drop=True,inplace=True)
df.loc[df['patent_id']=='6836905',:]

,patent_id,raw_assignee
2501741,6836905,Sande Hockey Inc.


In [21]:
df = df.iloc[0:2501741,:]
df.to_csv('E:/apps/pid_raw_assignee.csv',index=False)

__raw_assignee -> assignee__

In [30]:
os.chdir("E:/apps/db_table")

df = pd.read_csv('E:/apps/pid_raw_assignee.csv')

ass = pd.read_csv('assignee_raw.csv')
ass['raw_assignee'] = map(lambda x: x.upper(), ass['raw_assignee'])

len(ass)

187081

In [31]:
ass.drop_duplicates(inplace=True)
len(ass)

172725

In [33]:
df = df.merge(ass, how='left', on='raw_assignee')
len(df)

2501741

In [34]:
df.drop('raw_assignee', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
len(df)

2499501

In [37]:
df.dropna(inplace=True)
len(df)

1799943

In [38]:
df.to_csv('patent_assignee_1.csv',index=False)
df1 = pd.read_csv('patent_assignee_2.csv')

df2 = pd.concat([df,df1],ignore_index=True)

len(df2), len(df), len(df1)

(3142592, 1799943, 1342649)

In [39]:
df2.to_csv('patent_assignee.csv',index=False)

__nber, year__

In [1]:
import csv
import os

os.chdir('E:/apps')

raw = open('raw_uspto/historical_masterfile.csv')
reader = csv.reader(raw)
header = reader.next()
header

['appl_id',
 'pubno',
 'patent',
 'nber',
 'uspc',
 'uspc_sub',
 'appl_dt',
 'prior_dt',
 'pub_dt',
 'disp_dt',
 'disp_ty',
 'exp_dt',
 'exp_dt_max',
 'pta']

In [ ]:
IT = ['2','4']
YEAR = []   
for i in range(0,4):
    y=2001
    y += i
    YEAR.append(str(y))
    
header = ['patent_id','nber','uspc','uspc_sub','year']

with open('raw_uspto/historical_masterfile.csv','rb') as raw,open('patent_class_EE&IT_01_04.csv','wb') as sel, open('patent_class_BIO_01_04.csv','wb') as sel2:
    reader = csv.reader(raw)
    
    writer = csv.writer(sel)
    writer2 = csv.writer(sel2)
    
    writer.writerow(header)
    writer2.writerow(header)
    
    for line in reader:
        try:            
            if line[3][0] in IT and line[9][-4:] in YEAR:
                row = line[2:6] + [line[9][-4:]]
                writer.writerow(row)

            elif line[3][0] =='3' and line[9][-4:] in YEAR:
                row = line[2:6] + [line[9][-4:]]
                writer2.writerow(row)
                
        except ValueError:      #if pid != int, go to the next pat
                continue    
            

In [2]:
import os
import pandas as pd

os.chdir('E:/apps')

df = pd.read_csv('patent_class_BIO_01_04.csv')
df1 = pd.read_csv('patent_class_BIO_05.csv')

df2 = pd.concat([df,df1],ignore_index=True)

len(df2), len(df), len(df1)

(210087, 70863, 139224)

In [3]:
df2.to_csv('db_table/patent_class_BIO.csv',index=False)

In [4]:
df = pd.read_csv('patent_class_EE&IT_01_04.csv')
df1 = pd.read_csv('patent_class_EE&IT_05.csv')

df2 = pd.concat([df,df1],ignore_index=True)

df2.to_csv('db_table/patent_class_EE&IT.csv',index=False)

### 2. 기업간 adj matrix 제작
- 특정 산업에 해당하는 patent selection (patent_class, pat_citation_byfirm join)  
__=> patent_id, cited, year__  
- patent_id -> assignee -> gvkey (patent_assignee join, assignee_gvkey join)   
__=> gvkey, cited, year__
- cited -> assignee -> gvkey (patent_assignee join, assignee_gvkey join)  
__=> gvkey, gvkey_cited, year__
- year 기준으로 subset, 각각에 대해 adj matrix 제작 (여긴 R 사용)

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('E:/apps/db_table')

__1. patent selection__

In [2]:
df1 = pd.read_csv('pat_citation_byfirm_1.csv')
df2 = pd.read_csv('pat_citation_byfirm_2.csv')

df = pd.concat([df1,df2],ignore_index=True)
len(df), len(df1), len(df2)

(19694038, 6884027, 12810011)

In [3]:
del df1, df2

bio = pd.read_csv('patent_class_BIO.csv')
ee = pd.read_csv('patent_class_EE&IT.csv')

bio = bio.loc[:,['patent_id','year']]
ee = ee.loc[:,['patent_id','year']]

bio = bio.merge(df,how='left',on='patent_id')
ee = ee.merge(df,how='left',on='patent_id')

bio.dropna(how='any',inplace=True)  #drop patents which don't cite apps
ee.dropna(how='any',inplace=True)
print len(bio), len(ee)

3870228 8943754


In [4]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__2. patent_id -> gvkey__

In [7]:
df = pd.read_csv('patent_assignee.csv')

bio = bio.merge(df, how='left', on='patent_id')
ee = ee.merge(df, how='left', on='patent_id')

df = pd.read_csv('assignee_gvkey_BIO.csv')
df1 = pd.read_csv('assignee_gvkey_EE&IT.csv')

bio = bio.merge(df, how='left', on='assignee')
ee = ee.merge(df1, how='left', on='assignee')

bio = bio.loc[:,['gvkey','cited','year']]
ee = ee.loc[:,['gvkey','cited','year']]

len(bio), len(ee)

(3953630, 9056153)

In [8]:
bio.dropna(how='any',inplace=True)  #drop patents which don't have gvkey of assignee
ee.dropna(how='any',inplace=True)

len(bio), len(ee)

(1935285, 4331836)

In [9]:
ee.iloc[0:10,:]

,gvkey,cited,year
22,19349.0,6047552,2005
23,19349.0,5737922,2005
24,19349.0,5467815,2005
30,5047.0,5778676,2005
31,5047.0,5816049,2005
32,5047.0,5865024,2005
33,5047.0,6050082,2005
34,5047.0,6195607,2005
35,5047.0,6449953,2005
36,5047.0,6460340,2005


In [11]:
bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__3. cited -> gvkey_cited__

In [2]:
bio=pd.read_csv('bio_temp.csv')
ee=pd.read_csv('ee_temp.csv')

In [3]:
df = pd.read_csv('patent_assignee.csv')
df['patent_id'] = map(lambda x: str(x),df['patent_id'])

bio.rename(columns={'cited':'patent_id'}, inplace=True)
ee.rename(columns={'cited':'patent_id'}, inplace=True)

bio = bio.merge(df, how='left', on='patent_id')
ee = ee.merge(df, how='left', on='patent_id')

df = pd.read_csv('assignee_gvkey_BIO.csv')
df1 = pd.read_csv('assignee_gvkey_EE&IT.csv')

bio = bio.merge(df, how='left', on='assignee', suffixes=('', '_cited'))
ee = ee.merge(df1, how='left', on='assignee', suffixes=('', '_cited'))

bio = bio.loc[:,['gvkey','gvkey_cited','year']]
ee = ee.loc[:,['gvkey','gvkey_cited','year']]

len(bio), len(ee)

(1958232, 4387660)

In [4]:
bio.dropna(how='any',inplace=True)  #drop patents which cited gvkey-missing application
ee.dropna(how='any',inplace=True)

len(bio), len(ee)

(532649, 2425461)

In [5]:
bio['gvkey'] = map(int,bio['gvkey'])
ee['gvkey'] = map(int,ee['gvkey'])

bio.to_csv('E:/apps/pat_cit_edge_BIO.csv',index=False)
ee.to_csv('E:/apps/pat_cit_edge_EE&IT.csv',index=False)

### 3. weight sum
- proximity matrix 구하기
    - 전에 사용한 cls vec 사용
    - 전체 기업 중 appcit_matrix에 있는 기업만 고르기 (appcit_matrix의 0번째 col, class_vector join)
    - 함수 사용 matrix 제작
- adj, prox matrix 곱하기
    - weight sum > 0인 경우만 남기기 (patcit이 0 이하면 참여 안 한거지 아예)

__proximity__

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import math
from math import sqrt

os.chdir('E:/apps/patcit measure')

In [2]:
def prox_numer(l1,l2):
    cnt = 0.0
    for i in xrange(len(l1)):
        if l1[i]!=0 and l2[i]!=0:
            w= float(l1[i]) * l2[i]
            cnt += w
    
    return cnt 

def prox_denom(l1,l2):
    a = sqrt(sum(map(lambda x: x**2, l1)))
    b = sqrt(sum(map(lambda x: x**2, l2)))
    
    return a*b 

def proximity(l1,l2):
    try:
        return round(prox_numer(l1,l2) / prox_denom(l1,l2),3)
    
    except ZeroDivisionError:
        return 0.0          # no proximity = 0

def get_prox(l1,arr):
    sim = []
    for row in arr:
        sim.append(proximity(l1,row))
        
    return sim

In [3]:
def final(ind,y):
    gvkey = 'patcit_matrix_' + ind + '_' + str(y) +'.csv'
    clsvec = 'class_vector_' + ind + '_' + str(y) +'.csv'

    df = pd.read_csv(gvkey).iloc[:,0].to_frame()
    df1 = pd.read_csv(clsvec)
    
    df = df.merge(df1,how='left',on='gvkey')
    df.fillna(0, inplace=True)
    arr = df.values

    header = arr[:,0].tolist()
    header = ['gvkey'] + map(int,header)

    arr = np.delete(arr,0,1)
    
    fname = 'proximity_' + ind + '_' + str(y) +'.csv'
    with open(fname,'wb') as f:
        writer = csv.writer(f)
        writer.writerow(header)

        i=1  
        for row in arr:        
            line = get_prox(row,arr)
            line = [header[i]] + line             
            writer.writerow(line)
            i += 1

In [4]:
for y in range(2005,2013):
    final('bio',y)
    final('EE&IT',y)

__weight sum__

In [7]:
def get_patcit(ind,y):
    cit = 'patcit_matrix_' + ind + '_' + str(y) +'.csv'
    prox = 'proximity_' + ind + '_' + str(y) +'.csv'
    
    cit = pd.read_csv(cit)
    prox = pd.read_csv(prox)

    df = cit * prox

    df['pat_cit'] = 0.0
    
    for i in range(0,len(df)):
        df.loc[i,'gvkey'] = int(sqrt(df.loc[i,'gvkey']))
        df.loc[i,'pat_cit'] = sum(df.iloc[i,1:]) - df.iloc[i,i+1]
        
    df['year'] = y
    df = df.loc[:,['gvkey','year','pat_cit']]
    df = df.loc[df['pat_cit']>0,:]
    
    fname = 'patcit_' + ind + '_' + str(y) +'.csv'
    df.to_csv(fname,index=False)

In [8]:
for y in range(2005,2013):
    get_patcit('bio',y)
    get_patcit('EE&IT',y)

In [9]:
bio = pd.read_csv('patcit_bio_2005.csv')
ee = pd.read_csv('patcit_EE&IT_2005.csv')

for y in range(2006,2013):
    df = pd.read_csv('patcit_bio_' + str(y) +'.csv')
    bio = pd.concat([bio,df],ignore_index=True)
    
    df = pd.read_csv('patcit_EE&IT_' + str(y) +'.csv')
    ee = pd.concat([ee,df],ignore_index=True)
    
bio.sort_values(by=['gvkey','year'],inplace=True)
bio.reset_index(drop=True, inplace=True)
ee.sort_values(by=['gvkey','year'],inplace=True)
ee.reset_index(drop=True,inplace=True)

In [10]:
bio.to_csv('patcit_bio.csv',index=False)
ee.to_csv('patcit_EE&IT.csv',index=False)

### 4. patent stock
- 2001 ~ 2012까지 firm-year로 patent counting
- gvkey, year (patent_class, patent_assignee join)
- groupby
- 4년씩 더하는 fn 적용 -> gvkey, year, stock

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('E:/apps/db_table')

__groupby__

In [2]:
bio = pd.read_csv('patent_class_BIO.csv')
ee = pd.read_csv('patent_class_EE&IT.csv')

bio = bio.loc[:,['patent_id','year']]
ee = ee.loc[:,['patent_id','year']]

df = pd.read_csv('patent_assignee.csv')

bio = bio.merge(df, how='left', on='patent_id')
ee = ee.merge(df, how='left', on='patent_id')

df = pd.read_csv('assignee_gvkey_BIO.csv')
df1 = pd.read_csv('assignee_gvkey_EE&IT.csv')

bio = bio.merge(df, how='left', on='assignee')
ee = ee.merge(df1, how='left', on='assignee')

bio = bio.loc[:,['gvkey','year']]
ee = ee.loc[:,['gvkey','year']]

len(bio), len(ee)

(218383, 1178075)

In [3]:
bio.dropna(how='any',inplace=True)  #drop gvkey-missing patents
ee.dropna(how='any',inplace=True)

len(bio), len(ee)

(72384, 727747)

In [5]:
bio['count'] = 0
ee['count'] = 0

bio = bio.groupby(by=['gvkey','year'],as_index=False).count()
ee = ee.groupby(by=['gvkey','year'],as_index=False).count()

bio.to_csv('bio_temp.csv',index=False)
ee.to_csv('ee_temp.csv',index=False)

__aggregate__
- 모든 gvkey에 대해 2001 ~ 2012 count를 만듦 (없는 경우는 0으로 취급)
- 5년씩 sum
- 0인 경우 버림

In [52]:
def get_pstock(df):    
    gvkey = list(df['gvkey'].unique())
    year = list(df['year'].unique())
    
    a = []
    b = []
    
    for i in gvkey:
        for j in year:
            a.append(i)
            b.append(j)
            
    temp = pd.DataFrame({'gvkey':a,'year':b})
    
    temp = temp.merge(df,how='left',on=['gvkey','year'])
    temp.fillna(0, inplace=True)
            
    gk = []
    yy = list(temp['year'][temp['year']>2005])
    ps = []
    
    for k in gvkey:
        df1 = temp.loc[temp['gvkey']==k,:]
        
        for i in range(0,len(df1)-5):
            gk.append(k)            
            ps.append(sum(df1['count'][i:i+5]))
            
    pstock = pd.DataFrame({'gvkey':gk,'year':yy,'pstock':ps})
    pstock = pstock.loc[pstock['pstock'] > 0,:]
    
    return pstock

In [55]:
get_pstock(bio).to_csv('E:/apps/feature/patent_stock_bio.csv',index=False)
get_pstock(ee).to_csv('E:/apps/feature/patent_stock_EE&IT.csv',index=False)